In [1]:
import panel as pn
import param as pm
import hvplot.streamz
import pandas as pd
import numpy as np
from kappa_model import PAMMModel
pn.config.throttled = False

import matplotlib.pyplot as plt

# Access the default Matplotlib color cycle
default_colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

# Swapping the elements
default_colors[1], default_colors[3] = default_colors[3], default_colors[1]


In [2]:
volume_by_day = pd.read_csv('data/volume_agg_by_day.csv')
volume_by_day['match_datetime'] = pd.to_datetime(volume_by_day['match_datetime'])
volume_by_day['volume'] = volume_by_day['volume'] / 32.40

In [4]:

class ShockAbsorberApp(pm.Parameterized):
    # history = pm.DataFrame(precedence=-1)
    market_movement = pm.Number(0.001, label='Buy/Sell Pressure', bounds=(-0.05, 0.05), step=0.001)
    mint_fee = pm.Number(0.05, bounds=(0,0.15), step=0.001)
    burn_fee = pm.Number(0.05, bounds=(0,0.15), step=0.001)
    reserve_ratio = pm.Number(1/3, bounds=(0.2,1), step=0.001)
    initial_price = pm.Number(0.5, bounds=(0.01, None), precedence=-1)
    initial_reserve = pm.Integer(int(5.3*100_000_000/32.40), precedence=-1)
    initial_supply = pm.Integer(12_000_000, precedence=-1)
    n = pm.Integer(1000, label="Number of Simulation Steps")
    run_sim = pm.Action(lambda self: self._run_sim())
    sim_counter = pm.Integer(0, precedence=-1)

    def __init__(self, **params):
        super().__init__(**params)
        self.update_reserve()
        self.update_pamm()
        self.init_sim()
        self._run_sim()

    def update_pamm(self):
        self.pamm = PAMMModel(kappa=1/self.reserve_ratio, initial_reserve=self.initial_reserve, initial_supply=self.initial_supply)
        self.pamm.update_constants()

    @pm.depends()
    def init_sim(self):
        # self.pamm = PAMMModel(kappa=1/self.reserve_ratio, initial_reserve=self.initial_reserve, initial_supply=self.initial_supply)
        # self.pamm.update_constants()
        self.i = 0
        volume = volume_by_day.sample(1)['volume'].iloc[0]
        initial_state = pd.DataFrame({
            'market_movement': 0,
            'volume': volume,
            'closing_price_secondary': self.pamm.spot_price(), 
            'closing_price_primary': self.pamm.spot_price(),
            'mint_price':self.pamm.spot_price() * (1 + self.mint_fee),
            'burn_price':self.pamm.spot_price() * (1 - self.burn_fee),
            'supply': self.pamm.supply,
            'reserve': self.pamm.reserve,
            'arb_factor': 0,
            'trade_size': 0,
            'mint_fee': 0,
            'burn_fee': 0,
            'arb_profit': 0,
            'total_fees_collected': 0,
            }, index=[self.i])
        
        self.history = initial_state.copy(deep=True)

    def market_step(self, **kwargs):
        previous = self.history.iloc[-1]
        volume = volume_by_day.sample(1)['volume'].iloc[0]
        
        # Secondary Market
        scale = 20 / (self.mint_fee + self.burn_fee)
        market_movement = (np.random.random()-0.5+self.market_movement)/scale
        new_closing_price_secondary = previous['closing_price_secondary'] + market_movement

        # Mint and Burn Price
        mint_price = previous['closing_price_primary'] * (1 + self.mint_fee)
        burn_price = previous['closing_price_primary'] * (1 - self.burn_fee)
    
        # Default Scenario. Secondary is within bounds.
        arb_factor = abs(market_movement)
        trade_size = 0
        mint_fee = 0
        burn_fee = 0
        mint_amount = 0
        burn_amount = 0
        arb_profit=0
    
        # Secondary Market is above Primary. Mint on curve and sell in secondary
        if new_closing_price_secondary > mint_price:
            trade_size = self.pamm.reserve * arb_factor
            mint_fee = trade_size * self.mint_fee
            mint_amount = self.pamm.deltaS_for_deltaR(trade_size - mint_fee)
            self.pamm.update_supply_reserve(deltaS=mint_amount, deltaR=trade_size - mint_fee)
            new_closing_price_secondary = new_closing_price_secondary * (1-arb_factor)
            arb_profit = arb_factor * np.sqrt(self.pamm.reserve) * trade_size
            
    
        # Secondary Market is below Primary. Buy on secondary and burn on the curve
        elif new_closing_price_secondary < burn_price:
            arb_factor = abs(market_movement) 
            trade_size = self.pamm.reserve * arb_factor
            burn_fee = trade_size * self.burn_fee
            burn_amount = self.pamm.deltaS_for_deltaR(-(trade_size - burn_fee))
            self.pamm.update_supply_reserve(deltaS=burn_amount, deltaR=-(trade_size - burn_fee))
            new_closing_price_secondary = new_closing_price_secondary * (1+arb_factor)
            arb_profit = arb_factor * np.sqrt(self.pamm.reserve) * trade_size
    
    
        # New State
        self.i += 1
        state_update = pd.DataFrame({
            'market_movement': market_movement,
            'volume': volume,
            'closing_price_secondary': new_closing_price_secondary, 
            'closing_price_primary': self.pamm.spot_price(),
            'mint_price':mint_price,
            'burn_price':burn_price,
            'supply': self.pamm.supply,
            'reserve': self.pamm.reserve,
            'arb_factor': arb_factor,
            'trade_size': trade_size,
            'mint_fee': mint_fee,
            'burn_fee':  burn_fee,
            'arb_profit': arb_profit,
            'total_fees_collected': mint_fee + burn_fee + arb_profit
        }, index=[self.i])
    
        # Global Data
        self.history = pd.concat([self.history, state_update])
        
        return state_update
        
    def _run_sim(self):
        self.init_sim()
        for _ in range(self.n):
            self.market_step()
        self.sim_counter += 1


    @pm.depends('sim_counter')
    def view_markets(self):
        return self.history.hvplot(color=default_colors, xlabel='Timesteps', ylabel='Price',  y=['closing_price_secondary', 'burn_price', 'mint_price', 'closing_price_primary'], title='Primary and Secondary Markets')

    @pm.depends('sim_counter')
    def view_fees_collected(self):
        return self.history[['arb_profit', 'burn_fee', 'mint_fee']].cumsum().hvplot.area(color=default_colors, xlabel='Timesteps', ylabel='USD', stacked=True, title="Fees Collected", shared_axes=False)

    @pm.depends('sim_counter')
    def view_charts(self):
        return pn.Column(self.view_markets, self.view_fees_collected)

    @pm.depends('sim_counter')
    def view_table(self):
        return pd.DataFrame(self.history.head())

    @pm.depends('reserve_ratio', 'initial_supply', 'initial_price', watch=True)
    def update_reserve(self):
        self.initial_reserve = int(self.reserve_ratio * self.initial_supply * self.initial_price)

    @pm.depends('initial_reserve')
    def view_pamm_state(self):
        self.update_pamm()
        return self.pamm.print_state_message()
    
    @pm.depends()
    def view(self):
        return pn.Column(
            pn.Row(
            pn.Column(
                pn.panel(self),
                self.view_pamm_state,
            ), 
            self.view_charts
        ), 
                         self.view_table
                        )

app = ShockAbsorberApp()

app.view()

Column
    [0] Row
        [0] Column
            [0] Column(margin=(5, 10), name='ShockAbsorberApp')
                [0] StaticText(value='<b>ShockAbsorberApp</b>')
                [1] FloatSlider(end=0.05, name='Buy/Sell Pressure', start=-0.05, step=0.001, value=0.001)
                [2] FloatSlider(end=0.15, name='Mint fee', step=0.001, value=0.05)
                [3] FloatSlider(end=0.15, name='Burn fee', step=0.001, value=0.05)
                [4] FloatSlider(name='Reserve ratio', start=0.2, step=0.001, value=0.3333333333333333)
                [5] IntInput(name='Number of Simulation S..., value=1000)
                [6] Button(name='Run sim')
            [1] ParamMethod(method, _pane=Markdown, defer_load=False)
        [1] ParamMethod(method, _pane=Column, defer_load=False)
    [1] ParamMethod(method, _pane=DataFrame, defer_load=False)